In [24]:
from langchain_neo4j import Neo4jGraph, Neo4jVector

url = "bolt://localhost:7687"
username = "neo4j"
password = "21020180"

graph = Neo4jGraph(url=url, username=username, password=password, refresh_schema=False)

In [4]:
# Tìm kiếm bằng name dùng fulltext index cho tìm kiếm gần đúng
graph.query(
    "CREATE FULLTEXT INDEX studentNameIndex IF NOT EXISTS FOR (n:`Sinh viên`) ON EACH [n.name] OPTIONS { indexConfig: { `fulltext.analyzer`: 'standard', `fulltext.eventually_consistent`: true}};")
graph.query(
    "CREATE FULLTEXT INDEX lecturerNameIndex IF NOT EXISTS FOR (n:`Giảng viên`) ON EACH [n.name] OPTIONS { indexConfig: { `fulltext.analyzer`: 'standard', `fulltext.eventually_consistent`: true}};")
graph.query(
    "CREATE FULLTEXT INDEX courseVietnameseNameIndex IF NOT EXISTS FOR (n:`Học phần`) ON EACH [n.vietnameseName] OPTIONS { indexConfig: { `fulltext.analyzer`: 'standard', `fulltext.eventually_consistent`: true}};")
graph.query(
    "CREATE FULLTEXT INDEX documentTitleIndex IF NOT EXISTS FOR (n:`Tài liệu`) ON EACH [n.title] OPTIONS { indexConfig: { `fulltext.analyzer`: 'standard', `fulltext.eventually_consistent`: true}};")
graph.query(
    "CREATE FULLTEXT INDEX onlineDocumentTitleIndex IF NOT EXISTS FOR (n:`Tài liệu online`) ON EACH [n.title] OPTIONS { indexConfig: { `fulltext.analyzer`: 'standard', `fulltext.eventually_consistent`: true}};")

# Constraint cho Sinh viên
graph.query(
    "CREATE CONSTRAINT studentIdUnique IF NOT EXISTS FOR (s:`Sinh viên`) REQUIRE s.studentid IS UNIQUE;")
graph.query(
    "CREATE INDEX studentIdIndex IF NOT EXISTS FOR (s:`Sinh viên`) ON (s.studentid);")
graph.query(
    "CREATE INDEX studentDobIndex IF NOT EXISTS FOR (s:`Sinh viên`) ON (s.dob);")

# Constraint cho Giảng viên
graph.query(
    "CREATE CONSTRAINT lecturerIdUnique IF NOT EXISTS FOR (l:`Giảng viên`) REQUIRE l.id IS UNIQUE;")
graph.query(
    "CREATE INDEX lecturerIdIndex IF NOT EXISTS FOR (l:`Giảng viên`) ON (l.id);") 

# Constrain cho Học phần
graph.query(
    "CREATE CONSTRAINT courseIdUnique IF NOT EXISTS FOR (c:`Học phần`) REQUIRE c.id IS UNIQUE;")
graph.query(
    "CREATE INDEX courseIdIndex IF NOT EXISTS FOR (c:`Học phần`) ON (c.id);")

# Constraint cho Lớp học
graph.query(
    "CREATE CONSTRAINT classIdUnique IF NOT EXISTS FOR (c:`Lớp`) REQUIRE c.id IS UNIQUE;")
graph.query(
    "CREATE INDEX classIdIndex IF NOT EXISTS FOR (c:`Lớp`) ON (c.name);")

# Constraint cho Tình trạng học tập
graph.query(
    "CREATE CONSTRAINT academicStatusUnique IF NOT EXISTS FOR (a:`Tình trạng học tập`) REQUIRE a.status IS UNIQUE;")
graph.query(
    "CREATE INDEX academicStatusIndex IF NOT EXISTS FOR (a:`Tình trạng học tập`) ON (a.status);")

# Constraint cho Học phí
graph.query(
    "CREATE CONSTRAINT tuitionIdUnique IF NOT EXISTS FOR (t:`Học phí`) REQUIRE t.id IS UNIQUE;")
graph.query(
    "CREATE INDEX tuitionAmountIndex IF NOT EXISTS FOR (t:`Học phí`) ON (t.amount);")

# Constraint cho Tài liệu
graph.query(
    "CREATE CONSTRAINT documentIdUnique IF NOT EXISTS FOR (d:`Tài liệu`) REQUIRE d.id IS UNIQUE;")

# Constraint cho Tài liệu online
graph.query(
    "CREATE CONSTRAINT onlineDocumentIdUnique IF NOT EXISTS FOR (d:`Tài liệu online`) REQUIRE d.id IS UNIQUE;")

# Constraint cho Ngành học
graph.query(
    "CREATE CONSTRAINT majorIdUnique IF NOT EXISTS FOR (m:`Ngành học`) REQUIRE m.id IS UNIQUE;")
graph.query(
    "CREATE INDEX majorNameIndex IF NOT EXISTS FOR (m:`Ngành học`) ON (m.name);")

# Constraint cho Đơn vị
graph.query(
    "CREATE CONSTRAINT unitIdUnique IF NOT EXISTS FOR (u:`Đơn vị`) REQUIRE u.id IS UNIQUE;")
graph.query(
    "CREATE INDEX unitNameIndex IF NOT EXISTS FOR (u:`Đơn vị`) ON (u.name);")

[]

In [25]:
import re

def remove_lucene_chars(text: str) -> str:
    return re.sub(r'[+\-&|!(){}[\]^"~*?:\\]', ' ', text)

def generate_full_text_query_simple(input: str) -> str:
    cleaned_input = remove_lucene_chars(input).strip()
    # Có thể cần bao trong dấu nháy kép nếu analyzer của bạn xử lý phrase tốt hơn
    # return f'"{cleaned_input}"'
    # Hoặc để dạng đơn giản, dựa vào analyzer
    return cleaned_input
    # Hoặc thêm fuzziness nếu cần
    # return f'{cleaned_input}~1'

In [ ]:
from openai_interacter import OpenAIChatInterface
from pydantic import BaseModel, Field
from typing import List, Literal, Union

EntityType = Literal[
    "Sinh viên",
    "Mã SV",
    "Lớp",
    "Tình trạng học tập",
    "Giảng viên",
    "Chức danh"  
    "Học phần",
    "Tài liệu",
    "Tài liệu online",
    "Ngành học",
    "Đơn vị",  
    "TargetInfo",
    "Unknown"    
]

class EntityInfo(BaseModel):
    """Represents a single extracted entity with its type."""
    name: str = Field(..., description="The actual text corresponding to the entity (e.g., 'Đinh Thái Dương', '21020180', 'Hệ thống thông tin', 'Lập trình căn bản C++', 'Xác suất thống kê').")
    type: EntityType = Field(..., description="The type of the extracted entity.")

class StructuredEntities(BaseModel):
    """Structured information about entities extracted from text."""
    entities: List[EntityInfo] = Field(
        ...,
        description="A list of all identified entities, each with its name and type. "
                    "Includes main entities like students, studentId, classes, statuses, lectures, titles, courses, fees, documents, online documents, majors, units, and potentially the target information the user is asking about (e.g., if asking 'tình trạng của X?', 'tình trạng' could be a TargetInfo entity)."
    )
    
ENTITY_EXTRACTION_SYSTEM_PROMPT = """
You are an expert entity extraction system for a Vietnamese university knowledge graph.
Your goal is to identify entities from the user's query and classify them according to the specified types.
The Knowledge Graph contains information about lectures, courses, majors, students, classes, documents, online documents, academic statuses and units.

Entity Types:
- **Sinh viên**: A student's full name (e.g., "Đinh Thái Dương").
- **Mã SV**: A student's ID, typically a sequence of numbers (e.g., "21020190"). If the input text is clearly a student ID, use this type.
- **Lớp**: A class name (e.g., "K66I-CN", "K65C-CE1"). Often starts with 'K' followed by numbers and letters.
- **Tình trạng học tập**: An academic status description (e.g., "cảnh báo học vụ", "nhắc nhở kết quả chưa tốt").
- **Giảng viên**: A lecturer's full name (e.g., "Nguyễn Ngọc hoá").
- **Chức danh**: A lecturer's title or position (e.g., "Tiến Sĩ", "Thạc Sĩ").
- **Học phần**: A course name (e.g., "Cấu trúc dữ liệu", "Giải tích 1", "Xác suất thống kê").
 **Tài liệu**: A document title (e.g., "Giáo trình Cấu trúc dữ liệu").
- **Tài liệu online**: An online document title (e.g., "Tài liệu học tập trực tuyến").
- **Đơn vị**: An academic unit or department (e.g., "Trường Đhcn", "Trường ĐHKHXHNV").
- **Ngành học**: A major or field of study (e.g., "Công nghệ thông tin", "Hệ Thống Thông Tin").
- **TargetInfo**: Keywords indicating what specific information the user wants (e.g., "tình trạng", "lớp", "học phí", "điểm", "giảng viên", "tài liệu").

Example Queries and Outputs:
- Query 1: "Ai dạy môn Xác suất thống kê?"
Expected Entities:
[
  {"name": "Giảng viên", "type": "TargetInfo"},
  {"name": "Xác suất thống kê", "type": "Học phần"}
]

- Query 2: "Ai dạy môn Giải tích 2?"
Expected Entities:
[
  {"name": "Giảng viên", "type": "TargetInfo"},
  {"name": "Giải tích 2", "type": "Học phần"}
]

- Query 3: "Tình trạng học tập của sinh viên 21021509 là gì?"
Expected Entities:
[
  {"name": "Tình trạng học tập", "type": "TargetInfo"},
  {"name": "21021509", "type": "Mã SV"}
]

- Query 4: "Những sinh viên nào thuộc lớp K66I-CN?"
Expected Entities:
[
  {"name": "Sinh viên", "type": "TargetInfo"},
  {"name": "K66I-CN", "type": "Lớp"}
]

- Query 5: "Tài liệu môn Cấu trúc dữ liệu"
Expected Entities:
[
  {"name": "Tài liệu", "type": "TargetInfo"},
  {"name": "Cấu trúc dữ liệu", "type": "Học phần"}
]

- Query 6: "Tiến sĩ nào theo chuyên ngành Hệ thống thông tin?"
Expected Entities:
[
  {"name": "Tiến sĩ", "type": "Chức danh"},
  {"name": "Hệ thống thông tin", "type": "Ngành học"}
]

Instructions:
1. Analyze the user's query: "{question}".
2. Identify all relevant entities mentioned.
3. Classify each entity using **only** the types listed above.
4. If the user asks *about* something specific (like 'tình trạng' or 'lớp'), extract that keyword as type 'TargetInfo'.
5. For student IDs, use the type 'Mã SV'. For student names, use 'Sinh viên'.
6. Output the result as a JSON object matching the provided schema.
"""

In [189]:
parts = ENTITY_EXTRACTION_SYSTEM_PROMPT.split('Instructions:')
system_content = parts[0].strip()
system_content

'You are an expert entity extraction system for a Vietnamese university knowledge graph.\nYour goal is to identify entities from the user\'s query and classify them according to the specified types.\nThe Knowledge Graph contains information about lectures, courses, majors, students, classes, documents, online documents, academic statuses and units.\n\nEntity Types:\n- **Sinh viên**: A student\'s full name (e.g., "Đinh Thái Dương").\n- **Mã SV**: A student\'s ID, typically a sequence of numbers (e.g., "21020190"). If the input text is clearly a student ID, use this type.\n- **Lớp**: A class name (e.g., "K66I-CN", "K65C-CE1"). Often starts with \'K\' followed by numbers and letters.\n- **Tình trạng học tập**: An academic status description (e.g., "cảnh báo học vụ", "nhắc nhở kết quả chưa tốt").\n- **Giảng viên**: A lecturer\'s full name (e.g., "Nguyễn Ngọc hoá").\n- **Chức danh**: A lecturer\'s title or position (e.g., "Tiến Sĩ", "Thạc Sĩ").\n- **Học phần**: A course name (e.g., "Cấu trú

In [178]:
text = "Xin chào"
instructions_template = parts[1].strip()
user_content = instructions_template.replace("{question}", text)
user_content

'1. Analyze the user\'s query: "Xin chào".\n2. Identify all relevant entities mentioned.\n3. Classify each entity using **only** the types listed above.\n4. If the user asks *about* something specific (like \'tình trạng\' or \'lớp\'), extract that keyword as type \'TargetInfo\'.\n5. For student IDs, use the type \'Mã SV\'. For student names, use \'Sinh viên\'.\n6. Output the result as a JSON object matching the provided schema.'

In [209]:
from typing import List, Dict, Any, Optional, Type
# Function to process text and extract structured entities

def extract_entities(text: str) -> Optional[StructuredEntities]:
    # Sử dụng prompt mới, thay thế {question}
    parts = ENTITY_EXTRACTION_SYSTEM_PROMPT.split('Instructions:')
    system_content = parts[0].strip()
    instructions_template = parts[1].strip()
    user_content = instructions_template.replace("{question}", text)
    # user_content = ENTITY_EXTRACTION_SYSTEM_PROMPT.split('Instructions:')[1].split('Example Query:')[0].replace("{question}", text).strip()
    # system_content = ENTITY_EXTRACTION_SYSTEM_PROMPT.split('Instructions:')[0].strip()

    formatted_messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content} # Chỉ chứa phần query của người dùng
    ]

    print("Input messages for entity extraction: ", formatted_messages) # In ra để debug

    try:
        entity_llm = OpenAIChatInterface(model_name="gpt-4o-mini", temperature=0, initial_messages=formatted_messages)

        entity_llm.enable_structured_output(StructuredEntities)
        structured_output = entity_llm.parse_structured_output()
        print("Extracted entities:", structured_output) # In ra để debug
        return structured_output
    except Exception as e:
        print(f"Error during entity extraction: {e}")
        return StructuredEntities(entities=[])

In [101]:
def extract_entities_with_examples(text: str) -> Optional[StructuredEntities]:
    # --- Phân tích prompt gốc để tách system, instructions và examples ---

    # Tách phần system message và phần còn lại (instructions + examples)
    prompt_parts = ENTITY_EXTRACTION_SYSTEM_PROMPT.split('Instructions:')
    system_content_base = prompt_parts[0].strip() # Phần trước "Instructions:"

    # Tách phần instructions và các example blocks
    instruction_parts = prompt_parts[1].split('Example Query:')
    instructions = instruction_parts[0].strip() # Phần instructions có đánh số (1, 2,...)

    # Kết hợp lại thành system message hoàn chỉnh
    system_content = system_content_base + "\n\nInstructions:\n" + instructions

    # Tạo danh sách tin nhắn ví dụ
    example_messages = []
    # Duyệt qua từng block ví dụ (bắt đầu từ index 1 sau khi split)
    for example_block in instruction_parts[1:]:
        # Tách Query và Expected Entities
        parts = example_block.split('Expected Entities:')
        if len(parts) != 2:
            print(f"Cảnh báo: Bỏ qua block ví dụ bị lỗi định dạng: {example_block[:50]}...")
            continue

        query_line = parts[0].strip()
        expected_json_block = parts[1].strip()

        # Trích xuất câu hỏi từ dòng Query (xóa tiền tố "Example Query: " và dấu nháy nếu có)
        query_text = query_line.replace("Example Query:", "").strip()
        if query_text.startswith('"') and query_text.endswith('"'):
             query_text = query_text[1:-1] # Loại bỏ dấu nháy kép nếu có

        # Trích xuất chuỗi JSON từ block Expected Entities.
        # Sử dụng regex để tìm block JSON đầu tiên trông giống danh sách các object.
        # Đây là điểm có thể cần điều chỉnh nếu định dạng ví dụ thay đổi.
        json_match = re.search(r'\[\s*\{.*?\}\s*\]', expected_json_block, re.DOTALL)
        if not json_match:
             print(f"Cảnh báo: Không tìm thấy định dạng JSON trong block ví dụ: {example_block[:50]}...")
             continue

        json_string = json_match.group(0)

        # Thêm cặp tin nhắn User/Assistant (ví dụ) vào danh sách
        example_messages.append({"role": "user", "content": query_text})
        # Nội dung của assistant trong ví dụ chính là chuỗi JSON mong muốn
        example_messages.append({"role": "assistant", "content": json_string})

    # --- Xây dựng danh sách messages hoàn chỉnh cho API call ---
    formatted_messages = [{"role": "system", "content": system_content}]
    formatted_messages.extend(example_messages) # Thêm tất cả các ví dụ
    formatted_messages.append({"role": "user", "content": text}) # Thêm câu hỏi thực tế của người dùng

    print("Input messages for entity extraction: ", formatted_messages) # In ra để debug

    # --- Gọi API ---
    try:
        # Khởi tạo OpenAIChatInterface với toàn bộ lịch sử bao gồm system và examples
        # Nên dùng model mới nhất (gpt-4o) và có thể giảm temperature một chút để kết quả ổn định hơn
        entity_llm = OpenAIChatInterface(model_name="gpt-4o", temperature=0, initial_messages=formatted_messages)

        # Kích hoạt structured output - cần gọi sau khi đã set initial_messages
        entity_llm.enable_structured_output(StructuredEntities)

        # Gọi parse_structured_output. Hàm này sẽ dùng messages đã set
        structured_output = entity_llm.parse_structured_output()

        print("Extracted entities:", structured_output) # In ra để debug
        return structured_output
    except Exception as e:
        print(f"Lỗi trong quá trình trích xuất thực thể: {e}")
        # Có thể in thêm tin nhắn cuối cùng để debug phản hồi lỗi từ API nếu có
        # print(entity_llm.messages[-1])
        return StructuredEntities(entities=[])

In [210]:
entities = extract_entities("Giảng viên nào giảng dạy học phần Xác suất thống kê?")
entities

Input messages for entity extraction:  [{'role': 'system', 'content': 'You are an expert entity extraction system for a Vietnamese university knowledge graph.\nYour goal is to identify entities from the user\'s query and classify them according to the specified types.\nThe Knowledge Graph contains information about lectures, courses, majors, students, classes, documents, online documents, academic statuses and units.\n\nEntity Types:\n- **Sinh viên**: A student\'s full name (e.g., "Đinh Thái Dương").\n- **Mã SV**: A student\'s ID, typically a sequence of numbers (e.g., "21020190"). If the input text is clearly a student ID, use this type.\n\n- **Tình trạng học tập**: An academic status description (e.g., "cảnh báo học vụ", "nhắc nhở kết quả chưa tốt").\n- **Giảng viên**: A lecturer\'s full name (e.g., "Nguyễn Ngọc hoá").\n- **Chức danh**: A lecturer\'s title or position (e.g., "Tiến Sĩ", "Thạc Sĩ").\n- **Học phần**: A course name (e.g., "Cấu trúc dữ liệu", "Giải tích 1", "Xác suất thốn

StructuredEntities(entities=[EntityInfo(name='Giảng viên', type='TargetInfo'), EntityInfo(name='Xác suất thống kê', type='Giảng viên')])

In [212]:
entities = extract_entities("Ai dạy môn học Xác suất thống kê?")
entities

Input messages for entity extraction:  [{'role': 'system', 'content': 'You are an expert entity extraction system for a Vietnamese university knowledge graph.\nYour goal is to identify entities from the user\'s query and classify them according to the specified types.\nThe Knowledge Graph contains information about lectures, courses, majors, students, classes, documents, online documents, academic statuses and units.\n\nEntity Types:\n- **Sinh viên**: A student\'s full name (e.g., "Đinh Thái Dương").\n- **Mã SV**: A student\'s ID, typically a sequence of numbers (e.g., "21020190"). If the input text is clearly a student ID, use this type.\n\n- **Tình trạng học tập**: An academic status description (e.g., "cảnh báo học vụ", "nhắc nhở kết quả chưa tốt").\n- **Giảng viên**: A lecturer\'s full name (e.g., "Nguyễn Ngọc hoá").\n- **Chức danh**: A lecturer\'s title or position (e.g., "Tiến Sĩ", "Thạc Sĩ").\n- **Học phần**: A course name (e.g., "Cấu trúc dữ liệu", "Giải tích 1", "Xác suất thốn

StructuredEntities(entities=[EntityInfo(name='Giảng viên', type='TargetInfo'), EntityInfo(name='Xác suất thống kê', type='Tài liệu')])

In [ ]:
entities = extract_entities_with_examples("Giảng viên nào giảng dạy học phần Xác suất thống kê?")
entities

In [ ]:
entities = extract_entities_with_examples("Giảng viên nào giảng dạy học phần Giải tích 2?")
entities

In [111]:
def extract_entities(text: str, llm_interface: 'OpenAIChatInterface' = None) -> Optional[StructuredEntities]:
    if not llm_interface:
        llm_interface = OpenAIChatInterface(model_name="gpt-4o")
    
    llm_interface.messages = []  # Reset messages
    llm_interface.add_message("system", ENTITY_EXTRACTION_SYSTEM_PROMPT.format(question=text))
    llm_interface.enable_structured_output(StructuredEntities)
    
    try:
        structured_response = llm_interface.parse_structured_output()
        return structured_response
    except Exception as e:
        print(f"Error extracting entities: {e}")
        return None

In [213]:
def format_student_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    """
    Formats student-related Cypher query result record into a readable string.
    Filters based on wanted_info if provided.
    """
    parts = [f"Thông tin Sinh viên: {record.get('studentName')} (Mã SV: {record.get('studentId')})"]

    if record.get('dob'): parts.append(f"  - Ngày sinh: {record['dob']}")
    
    if record.get('className'): # Nếu query trả về className từ node Lớp
         parts.append(f"  - Lớp: {record['className']}")
    elif record.get('classProperty'): # Nếu query trả về thuộc tính class của Sinh viên
         parts.append(f"  - Lớp: {record['classProperty']}") # Giả sử tên prop là 'class'

    # Lưu ý property name: status (trong node Tình trạng học tập)
    # Query trả về academicStatus
    if (not wanted_info or 'tình trạng' in wanted_info) and record.get('academicStatus'):
         parts.append(f"  - Tình trạng học tập: {record['academicStatus']}")
    if (not wanted_info or 'ngành' in wanted_info) and record.get('majorName'):
         parts.append(f"  - Ngành học: {record['majorName']}")
    if (not wanted_info or 'học phí' in wanted_info) and record.get('tuitionAmount') is not None:
         parts.append(f"  - Học phí: {record['tuitionAmount']:,.0f} Đ")
         
    return "\n".join(parts)

def format_student_list_for_llm(students: List[Dict[str, Any]], context: str) -> str:
    """ Formats a list of students for a given context (class or status). """
    if not students:
        return f"Không tìm thấy sinh viên nào {context}."

    parts = [f"Danh sách sinh viên {context}:"]
    for student in students:
         # Lưu ý: KG của bạn dùng 'studentid' và 'name'
        student_info = f"- {student.get('name', 'N/A')}"
        if student.get('studentid'): # Dùng đúng tên property từ KG
             student_info += f" (Mã SV: {student.get('studentid')})"
        parts.append(student_info)
    return "\n".join(parts)

def format_lecturer_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    parts = [f"Thông tin Giảng viên: {record.get('lecturerName')} (ID: {record.get('lecturerId')})"]
    if record.get('title'):
        parts.append(f"  - Chức danh: {record['title']}")
    if record.get('courses'):
        parts.append(f"  - Môn dạy: {', '.join(record['courses'])}")
    return "\n".join(parts)

def format_course_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    parts = [f"Thông tin Học phần: {record.get('courseName')} (ID: {record.get('courseId')})"]
    if record.get('documents'):
        parts.append(f"  - Tài liệu: {', '.join(record['documents'])}")
    if record.get('onlineDocuments'):
        parts.append(f"  - Tài liệu online: {', '.join(record['onlineDocuments'])}")
    return "\n".join(parts)

def format_document_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    parts = [f"Thông tin Tài liệu: {record.get('title')} (ID: {record.get('documentId')})"]
    if record.get('authors'):
        parts.append(f"  - Tác giả: {record['authors']}")
    if record.get('publisherInfo'):
        parts.append(f"  - Nhà xuất bản: {record['publisherInfo']}")
    return "\n".join(parts)

def format_online_document_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    parts = [f"Thông tin Tài liệu online: {record.get('title')} (ID: {record.get('onlineDocId')})"]
    if record.get('link'):
        parts.append(f"  - Link: {record['link']}")
    return "\n".join(parts)

def format_tuition_info_for_llm(record: Dict[str, Any], wanted_info: Optional[str] = None) -> str:
    parts = [f"Thông tin Học phí: {record.get('amount'):,.0f} Đ (ID: {record.get('tuitionId')})"]
    return "\n".join(parts)

In [214]:
def structured_retriever(query: str) -> str:
    """
    Retrieves information from the graph based on extracted entities,
    focusing on Sinh viên, Lớp, Tình trạng học tập nodes and relationships.
    """
    results = []
    # 1. Extract structured entities
    entities_data: Optional[StructuredEntities] = extract_entities_with_examples(text=query)

    if not entities_data or not entities_data.entities:
        print("[Retriever] No entities extracted or error during extraction.")
        return "Không thể phân tích câu hỏi của bạn để tìm thực thể."

    # Tách thực thể chính và thông tin muốn hỏi
    main_entities = [e for e in entities_data.entities if e.type != 'TargetInfo']
    target_info_entities = [e for e in entities_data.entities if e.type == 'TargetInfo']
    wanted_info = target_info_entities[0].name if target_info_entities else None # Ví dụ: "tình trạng", "lớp"

    print(f"[Retriever] Main entities: {main_entities}, Wanted info: {wanted_info}") # Debug

    # 2. Query Graph based on extracted entities
    for entity in main_entities:
        cypher_query = ""
        params = {}
        result_formatter = None # Hàm để định dạng kết quả

        # --- Xử lý Sinh viên hoặc Mã SV ---
        if entity.type == 'Sinh viên' or entity.type == 'Mã SV':
            is_student_id_search = entity.type == 'Mã SV' or (entity.name.isdigit() and len(entity.name) > 5) # Heuristic

            if is_student_id_search:
                # Tìm bằng Mã SV (dùng index B-Tree)
                # Lưu ý: Property trong KG là 'studentid' (lowercase)
                print(f"[Retriever] Querying by student ID: {entity.name}")
                cypher_query = """
                MATCH (student:`Sinh viên` {studentid: $student_id})
                OPTIONAL MATCH (student)-[:CÓ_TÌNH_TRẠNG]->(status_node:`Tình trạng học tập`)
                OPTIONAL MATCH (student)-[:THUỘC_LỚP]->(class_node:`Lớp`)
                OPTIONAL MATCH (student)-[:CẦN_NỘP]->(tuition_node:`Học phí`)
                RETURN
                  student.studentid AS studentId, // Trả về 'studentId' để thống nhất output
                  student.name AS studentName,
                  student.dob AS dob,
                  class_node.name AS className,
                  status_node.status AS academicStatus,
                  tuition_node.amount AS tuitionAmount
                LIMIT 1
                """
                params = {"student_id": entity.name}
                result_formatter = format_student_info_for_llm
            else:
                # Tìm bằng Tên Sinh viên (dùng full-text index 'studentNameIndex' trên prop 'name')
                 print(f"[Retriever] Querying by student name: {entity.name}")
                 search_query = generate_full_text_query_simple(entity.name) # Tạo query full-text
                 cypher_query = """
                 CALL db.index.fulltext.queryNodes('studentNameIndex', $search_query, {limit: 5}) YIELD node AS student, score
                 OPTIONAL MATCH (student)-[:CÓ_TÌNH_TRẠNG]->(status_node:`Tình trạng học tập`)
                 OPTIONAL MATCH (student)-[:THUỘC_LỚP]->(class_node:`Lớp`)
                 OPTIONAL MATCH (student)-[:CẦN_NỘP]->(tuition_node:`Học phí`)
                 RETURN
                   student.studentid AS studentId,
                   student.name AS studentName,
                   student.dob AS dob,
                   class_node.name AS className,
                   status_node.status AS academicStatus,
                   tuition_node.amount AS tuitionAmount
                 """
                 params = {"search_query": search_query}
                 result_formatter = format_student_info_for_llm 

        # --- Xử lý Giảng viên ---
        elif entity.type == 'Giảng viên':
            search_query = generate_full_text_query_simple(entity.name)
            cypher_query = """
            CALL db.index.fulltext.queryNodes('lecturerNameIndex', $search_query, {limit: 5}) YIELD node AS lecturer, score
            OPTIONAL MATCH (lecturer)-[:GIẢNG_DẠY]->(course_node:`Học phần`)
            RETURN
              lecturer.id AS lecturerId,
              lecturer.name AS lecturerName,
              lecturer.title AS title,
              collect(course_node.vietnamesename) AS courses
            """
            params = {"search_query": search_query}
            result_formatter = format_lecturer_info_for_llm

        # --- Xử lý Học phần ---
        elif entity.type == 'Học phần':
            search_query = generate_full_text_query_simple(entity.name)
            cypher_query = """
            CALL db.index.fulltext.queryNodes('courseVietnameseNameIndex', $search_query, {limit: 5}) YIELD node AS course, score
            OPTIONAL MATCH (course)<-[:LÀ_TÀI_LIỆU_CHO]-(document_node:`Tài liệu`)
            OPTIONAL MATCH (course)<-[:LÀ_TÀI_LIỆU_TRỰC_TUYẾN_CHO]-(online_doc_node:`Tài liệu online`)
            RETURN
              course.id AS courseId,
              course.vietnamesename AS courseName,
              collect(document_node.title) AS documents,
              collect(online_doc_node.title) AS onlineDocuments
            """
            params = {"search_query": search_query}
            result_formatter = format_course_info_for_llm

        # --- Xử lý Tài liệu ---
        elif entity.type == 'Tài liệu':
            search_query = generate_full_text_query_simple(entity.name)
            cypher_query = """
            CALL db.index.fulltext.queryNodes('documentTitleIndex', $search_query, {limit: 5}) YIELD node AS document, score
            RETURN
              document.id AS documentId,
              document.title AS title,
              document.authors AS authors,
              document.publisherinfo AS publisherInfo
            """
            params = {"search_query": search_query}
            result_formatter = format_document_info_for_llm
            
        # --- Xử lý Tài liệu online ---
        elif entity.type == 'Tài liệu online':
            search_query = generate_full_text_query_simple(entity.name)
            cypher_query = """
            CALL db.index.fulltext.queryNodes('onlineDocumentTitleIndex', $search_query, {limit: 5}) YIELD node AS onlineDoc, score
            RETURN
              onlineDoc.id AS onlineDocId,
              onlineDoc.title AS title,
              onlineDoc.link AS link
            """
            params = {"search_query": search_query}
            result_formatter = format_online_document_info_for_llm
        
        # --- Xử lý Học phí (Nếu tìm bằng mã sinh viên hoặc số tiền) ---
        elif entity.type == 'Học phí':
            try:
                amount = float(entity.name.replace('.', '').replace(',', ''))
                cypher_query = """
                MATCH (tuition_node:`Học phí` {amount: $amount})
                RETURN
                  tuition_node.id AS tuitionId,
                  tuition_node.amount AS amount
                """
                params = {"amount": amount}
                result_formatter = format_tuition_info_for_llm
            except ValueError:
                results.append(f"Không thể chuyển đổi '{entity.name}' thành số tiền hợp lệ.")
                
        # --- Xử lý Lớp ---
        elif entity.type == 'Lớp':
            # Tìm node Lớp (giả sử tìm bằng tên/id lớp)
            # Lưu ý: property của Lớp là 'id' hoặc 'name' (viết hoa?)
            # Chúng ta sẽ tìm bằng cách khớp chính xác (case-sensitive nếu không xử lý)
             print(f"[Retriever] Querying by class name: {entity.name}")
             # Cần làm rõ property nào của Lớp dùng để tìm kiếm ('id' hay 'name') và kiểu viết hoa
             # Giả sử tìm bằng 'name' và giữ nguyên kiểu viết hoa người dùng nhập
             cypher_query = """
             MATCH (class_node:`Lớp`)
             WHERE class_node.name = $class_name // Hoặc class_node.id = $class_name, chú ý case
             // Tìm tất cả sinh viên thuộc lớp này
             MATCH (student:`Sinh viên`)-[:THUỘC_LỚP]->(class_node)
             RETURN collect({studentid: student.studentid, name: student.name}) AS students // Thu thập danh sách SV
             """
             params = {"class_name": entity.name}
             # Hàm format đặc biệt cho danh sách SV
             result_formatter = lambda record, wi: format_student_list_for_llm(record.get('students', []), f"thuộc lớp {entity.name}")


        # --- Xử lý Tình trạng học tập ---
        elif entity.type == 'Tình trạng học tập':
            # Tìm node Tình trạng học tập (dùng index trên 'status')
            # Lưu ý: property là 'status' (không viết hoa chữ cái đầu)
             print(f"[Retriever] Querying by academic status: {entity.name}")
             # Tìm bằng khớp chính xác trên thuộc tính 'status' (dùng index sẽ nhanh)
             cypher_query = """
             MATCH (status_node:`Tình trạng học tập` {status: $status_name}) // Dùng đúng tên property 'status'
             // Tìm tất cả sinh viên có tình trạng này
             MATCH (student:`Sinh viên`)-[:CÓ_TÌNH_TRẠNG]->(status_node)
             RETURN collect({studentid: student.studentid, name: student.name}) AS students // Thu thập danh sách SV
             """
             params = {"status_name": entity.name}
             # Hàm format đặc biệt cho danh sách SV
             result_formatter = lambda record, wi: format_student_list_for_llm(record.get('students', []), f"có tình trạng '{entity.name}'")

        # --- Thực thi Query và Định dạng Kết quả ---
        if cypher_query and result_formatter:
            try:
                print(f"[Retriever] Executing Cypher: {cypher_query} with params: {params}") # Debug
                response = graph.query(cypher_query, params)
                print(f"[Retriever] Cypher response: {response}") # Debug

                if not response:
                    results.append(f"Không tìm thấy thông tin cho {entity.type} '{entity.name}'.")
                else:
                     # Xử lý kết quả (có thể là 1 record hoặc list record)
                     for record in response:
                         formatted_record = result_formatter(record, wanted_info)
                         if formatted_record: # Chỉ thêm nếu có nội dung
                              results.append(formatted_record)

            except Exception as e:
                print(f"[Retriever] Error querying graph for entity '{entity.name}' (type: {entity.type}): {e}")
                results.append(f"Lỗi khi truy vấn thông tin cho {entity.type} '{entity.name}'.")
        elif not cypher_query:
             pass # Đã in ra unhandled type ở trên
        elif not result_formatter:
             print(f"[Retriever] No formatter defined for entity type {entity.type}")


    # 3. Tổng hợp kết quả
    if not results:
        return "Không tìm thấy thông tin liên quan trong Knowledge Graph."
    else:
        # Nối các kết quả từ các thực thể khác nhau (nếu có)
        return "\n\n".join(results)

In [216]:
# ans = structured_retriever("Lấy thông tin của Giảng viên Nguyễn Ngọc Hoá")
ans = structured_retriever("Ai dạy môn Xác suất thông kê?")
ans 

Input messages for entity extraction:  [{'role': 'system', 'content': 'You are an expert entity extraction system for a Vietnamese university knowledge graph.\nYour goal is to identify entities from the user\'s query and classify them according to the specified types.\nThe Knowledge Graph contains information about lectures, courses, majors, students, classes, documents, online documents, academic statuses and units.\n\nEntity Types:\n- **Sinh viên**: A student\'s full name (e.g., "Đinh Thái Dương").\n- **Mã SV**: A student\'s ID, typically a sequence of numbers (e.g., "21020190"). If the input text is clearly a student ID, use this type.\n\n- **Tình trạng học tập**: An academic status description (e.g., "cảnh báo học vụ", "nhắc nhở kết quả chưa tốt").\n- **Giảng viên**: A lecturer\'s full name (e.g., "Nguyễn Ngọc hoá").\n- **Chức danh**: A lecturer\'s title or position (e.g., "Tiến Sĩ", "Thạc Sĩ").\n- **Học phần**: A course name (e.g., "Cấu trúc dữ liệu", "Giải tích 1", "Xác suất thốn

'Không tìm thấy thông tin liên quan trong Knowledge Graph.'

In [34]:
from AugmentedLLM.llm import AugmentedLLM, LLMProvider
import json

# Create an AugmentedLLM instance with Anthropic provider
llm = AugmentedLLM(
    system_prompt="You are a helpful assistnat that uses the data they can get from a graph database to answer the user's question.",
    provider=LLMProvider.OPENAI,
    model_name="gpt-4o-mini",
    temperature=0.85, 
    max_tokens=4096, 
    use_react=True,
    debug_tools=True,
    debug_tokens=True,
    debug_messages=True
)

In [35]:
llm.add_tool(
    name="structured_retriever",
    description="A tool to retrieve structured information from a Neo4j graph database. It uses entity extraction to identify relevant entities and then queries the graph for detailed information.",
    input_schema={
            "query": {
                "type": "string",
                "description": "Can be a question or phrase indicating what information you want to get from the knowledge graph. Try to mention the specific entities (people, documents) etc you want information about.", 
                "required": True
            }
        },
    handler= structured_retriever
)

In [ ]:
prompt = "Ai dạy môn Xác Suất Thống Kê?"

# Print the response while filtering out tool/debug messages
for chunk in llm.generate(prompt):
    if isinstance(chunk, str):
        # Skip tool-related outputs and debug messages
        if any(marker in chunk for marker in [
            "[Tool Result]",
            "[Tool Use Started]", 
            "[Tool Input]",
            "[Stream Started]",
            "[Message Complete]",
            "[Stop Reason]",
            "[Debug]",
            "[Continuing conversation"
        ]):
            continue
        print(chunk, end="", flush=True)